In [21]:
import json
import os
import time
import re
import pandas as pd
import string


In [22]:

def twitter_feature_selection(json_file):
    selected_feature = {}
    selected_feature["text"] = json_file["text"]
    # selected_feature["id"] = json_file["id"]
    selected_feature["created_at"] = json_file["created_at"]
    # selected_feature['like_count'] = json_file['public_metrics']['like_count']
    selected_feature['retweet_count'] = json_file['public_metrics']['retweet_count']

    # selected_feature["hashtag"] = re.search(r'(\w+)+', json_file["text"])
    return selected_feature

def test_feature_selection(json_file):
    selected_feature = {}
    selected_feature["text"] = json_file["text"]
    selected_feature["created_at"] = json_file["created_at"]
    # selected_feature['retweet_count'] = json_file['retweet_count']

    # selected_feature["id"] = json_file["id"]
    return selected_feature

def convert_label(label):
    if label == "rumour":
        return 1
    elif label == "nonrumour":
        return 0 
def merge_dictionary_list(dict_list):
      return {
    k: [d.get(k) for d in dict_list if k in d]
    for k in set().union(*dict_list)
  } 
def remove_unwanted_keys(dict_list, keys):
    for item in dict_list:
        item.pop(keys, None)
    return dict_list

In [23]:
def load_data(file_name):
    total_json = []
    finished_dict = []
    with open("/Users/yuweng/Desktop/task1/" + file_name +".data.txt", "r") as f:
        file_name_event = f.readlines()      
    for event_str in file_name_event:
        event_list = event_str.rstrip().split(",")
        event_json = []
        path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + event_list[0] + ".json"
        # print(path)
        # print(os.path.exists(path))
        if os.path.exists(path):
            for id in event_list:
                path = "/Users/yuweng/Desktop/task1/" + file_name +'/' + id + ".json"
                # print(path)
                # print(os.path.exists(path))
                if os.path.exists(path):
                    related_tweet = json.load(open(path, "r"))
                    # print(related_tweet)
                    event_json.append(test_feature_selection(related_tweet))
            event_json = sorted(event_json, key=lambda x: time.mktime(time.strptime(x["created_at"], '%Y-%m-%dT%H:%M:%S.%fZ')))
            # print(event_json)
            merged_dict = merge_dictionary_list(event_json)
            # print(merged_dict)
            total_json.append(merged_dict)
            # print(total_json)
            finished_dict = remove_unwanted_keys(total_json, "created_at")
    return pd.DataFrame(finished_dict)

Load the Required data set

In [24]:
# df_train = load_data("train")
# df_dev = load_data("dev")
df_test = load_data("covid")

In [25]:
df_test

,text
0,"[According to the New York Times, Warner Bros...."
1,[Hurricane Hanna has made landfall in Texas.\n...
2,[Monkeys on the loose in India with stolen cor...
3,"[“If Trump felt comfortable having it here, th..."
4,[DISTURBING: Alabama officials say some studen...
...,...
15958,[I wonder how many lives could’ve been saved i...
15959,[The @thetimes front page on 17th March. The f...
15960,[Trump just completed the racism trifecta in a...
15961,[Here are a few of my photographs from today’s...


In [26]:
def text_preprocessing(tweet):
    r1 = r'https?:/\/\S+'
    ## remove @people
    r2 = r'@[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…？“”‘’！\\]+'
    ##remove tag
    r3 = r'#[a-zA-Z0-9’!"#$%&\'()*+,-./:;<=>?@，。?★、…？“”‘’！\\]+'

    tweet = tweet.encode('utf-8', 'replace').decode('utf-8')
    tweet = re.sub(r1, "", tweet)
    tweet = re.sub(r2, "", tweet)
    tweet = re.sub(r3, "", str(tweet).lower().strip())
    return tweet
def give_emoji_free_text(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F" 
        u"\U0001F300-\U0001F5FF" 
        u"\U0001F680-\U0001F6FF" 
        u"\U0001F1E0-\U0001F1FF"
        u"\U0001F1F2-\U0001F1F4"  
        u"\U0001F1E6-\U0001F1FF" 
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)
def remove_html(text):
    html = re.compile(r'<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    return re.sub(html, '', text)
def remove_punct(text):
    table = str.maketrans('', '', string.punctuation)
    return text.translate(table).strip()


Join the Text together and do the data preprocessing

In [27]:
# df_dev['text'][15]

In [28]:
# df_train['text'] = df_train.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
# df_train['text'] = df_train['text'].apply(lambda x:x.lower().replace('\n',''))
# df_train['text'] = df_train['text'].apply(lambda x:give_emoji_free_text(x))
# df_train['label'] = df_train.apply(lambda x:x["label"][0],axis=1)
# df_train['label'] = df_train['label'].astype('int')
# df_train['retweet_count'] = df_train.apply(lambda x:sum(x["retweet_count"]),axis=1)

# df_dev['text'] = df_dev.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
# df_dev['text'] = df_dev['text'].apply(lambda x:x.lower().replace('\n',''))
# df_dev['text'] = df_dev['text'].apply(lambda x:give_emoji_free_text(x))
# df_dev['label'] = df_dev.apply(lambda x:x["label"][0],axis=1)
# df_dev['label'] = df_dev['label'].astype('int')
# df_dev['retweet_count'] = df_dev.apply(lambda x:sum(x["retweet_count"]),axis=1)

df_test['text'] = df_test.apply(lambda x:text_preprocessing(''.join(x["text"])),axis=1)
df_test['text'] = df_test['text'].apply(lambda x:x.lower().replace('\n',''))
df_test['text'] = df_test['text'].apply(lambda x:give_emoji_free_text(x))
# df_test['retweet_count'] = df_test.apply(lambda x:sum(x["retweet_count"]),axis=1)







# df_train['text'] = df_train['text'].apply(lambda x:remove_URL(x))

# df_train['text'] = df_train['text'].apply(lambda x:remove_html(x))
# df_train['text'] = df_train['text'].apply(lambda x:remove_punct(x))

# df_dev['text'] = df_dev['text'].apply(lambda x:remove_URL(x))

# df_dev['text'] = df_dev['text'].apply(lambda x:remove_html(x))
# df_dev['text'] = df_dev['text'].apply(lambda x:remove_punct(x))

# df_test['text'] = df_test['text'].apply(lambda x:remove_URL(x))

# df_test['text'] = df_test['text'].apply(lambda x:remove_html(x))
# df_test['text'] = df_test['text'].apply(lambda x:remove_punct(x))

# df_train = df_train.dropna(axis=0, how='any')
# df_dev = df_dev.dropna(axis=0, how='any')
# df_test = df_test.dropna(axis=0, how='any')

In [29]:
df_test['text'][15]

'america has made incredible strides despite what corrupt career politicians are trying to do to this presidency and to our country.  deaths from new coronavirus are expected to reach 113,000 in the united states in mid-june, according to a new projection based on nine epidemiological models and released on tuesday. make america great again trump diffusion pharmaceutical(dffn)573% upside!!!!. now in talks with the fda for covid-19 treatment! oh yeah, highest unemployment numbers in the history of the world, most  cases, most  deaths (more than rest of works)! so what are corrupt politicians doing?    imaginary diversion from   epic fail  death-count 5/19 in countries that took prompt, aggressive action:s. korea: 263japan: 749hong kong: 4taiwan: 7nz: 21au: 100the us where  called covid-19 a hoax &amp; ignored it for 70 days, faced huge deficit of ventilators/masks/ppesus covid-19 death toll tops 92,133  mr. president do you know that electricity shock can kill a virus in a human body bu

In [30]:
df_test

,text
0,"according to the new york times, warner bros. ..."
1,hurricane hanna has made landfall in texas.the...
2,monkeys on the loose in india with stolen coro...
3,"“if trump felt comfortable having it here, the..."
4,disturbing: alabama officials say some student...
...,...
15958,i wonder how many lives could’ve been saved if...
15959,the front page on 17th march. the first line ...
15960,trump just completed the racism trifecta in a ...
15961,here are a few of my photographs from today’s ...


In [31]:
df_test.to_csv('covid.csv', index = False)

In [32]:
df_test = pd.read_csv('/Users/yuweng/Desktop/task1/covid.csv')

In [33]:
df_test.isnull().any()

text    True
dtype: bool

In [34]:
is_NaN = df_test.isnull()
row_has_NaN = is_NaN.any(axis=1)
rows_with_NaN = df_test[row_has_NaN]

In [35]:
rows_with_NaN

,text
2545,NaN
3585,NaN
4649,NaN
8890,NaN
9396,NaN
12562,NaN
15123,NaN
15887,NaN


In [36]:
# df_test.dropna(axis=0, how='any', inplace=True)
df_test.isnull().any()

text    True
dtype: bool

In [37]:
# df_test.to_csv('covid.csv',index= False)